In [7]:
import os
import re
import numpy as np
import pandas as pd
from argparse import ArgumentParser
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import ast

def cosine_similarity(vec_a, vec_b):
    """计算余弦相似度"""
    return np.dot(vec_a, vec_b) / (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))

def parse_embedding(embedding_str):
    """
    将存储在 CSV 中的嵌入字符串转换为 numpy 数组。
    假设嵌入存储为形如 "[0.1, 0.2, ...]" 的字符串。
    """
    # 使用 ast.literal_eval 将字符串转换为列表，再转换为 numpy 数组
    return np.array(ast.literal_eval(embedding_str))

class DocumentExtractor:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        """
        初始化本地嵌入模型。
        默认使用 SentenceTransformer 的 "all-MiniLM-L6-v2" 模型。
        """
        self.model = SentenceTransformer(model_name)
        
    def _get_embedding(self, text):
        """使用本地模型获取文本的嵌入向量"""
        # SentenceTransformer returns a numpy array
        return self.model.encode(text)
    
    def _cosine_similarity(self, vec_a, vec_b):
        """计算余弦相似度"""
        return np.dot(vec_a, vec_b) / (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
    
    def extract(self, entity, document_path):
        similarities = []
        entity_embedding = self.model.encode(entity, show_progress_bar=False)
        df = pd.read_csv(document_path)    
        for idx, row in df.iterrows():
            # 注意：如果 embedding 列已经是列表对象，则可以直接转换为 np.array
            # 但通常在 CSV 中会存储为字符串，所以我们需要解析它。
            row_embedding = parse_embedding(row['embedding'])
            sim = cosine_similarity(entity_embedding, row_embedding)
            similarities.append(sim)

        df["similarity"] = similarities
        top_n = df.nlargest(3, "similarity")
        return top_n
    
    def extract_and_save_embeddings(self, document_path, output_path):
        """
        针对 CSV 文件，计算每一行的嵌入向量（对所有列除去指定的 exclude_column）并将嵌入结果保存到新的 CSV 文件中。
        """
        df = pd.read_csv(document_path)
        embeddings = []
        # 计算每一行的嵌入：拼接所有非 exclude_column 的内容
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="计算行嵌入"):
            text = " ".join([str(row[col]) for col in df.columns if col =='column_description' or col=='注释']).strip()
            emb = self._get_embedding(text)
            embeddings.append(emb)
        
        # 将嵌入存入新列，注意这里嵌入是一个 numpy 数组，可以将其转换为列表
        df["embedding"] = [emb.tolist() for emb in embeddings]
        df.to_csv(output_path, index=False)
        print(f"新 CSV 文件已保存到 {output_path}")


extractor = DocumentExtractor('moka-ai/m3e-base')
extractor.extract_and_save_embeddings('data_dictionary.csv', 'data_dictionary_emb.csv')

计算行嵌入: 100%|██████████| 3489/3489 [00:51<00:00, 68.14it/s]


新 CSV 文件已保存到 data_dictionary_emb.csv


In [8]:
entities = ['中文全称', '全称变更', 'A股简称','法人','法律顾问','会计师事务所','董秘', '实控人', '近一个月最高价', '现金流量净额',
            '注册邮箱', '注册地址', '信披网址', '公司电话','硕士及以上学历（硕士+博士）的人员占比', '一级行业', 
            '唐山港集团股份有限公司是什么时间上市的']
for entity in entities:
    top_n = extractor.extract(entity, 'data_dictionary_emb.csv')
    print(f"与实体 '{entity}' 相关的table & column：")
    print(top_n[['table_name', 'column_name', 'column_description']])
    print("\n")


与实体 '中文全称' 相关的table & column：
            table_name column_name column_description
37    LC_StockArchives     ChiName               中文名称
61       LC_NameChange     ChiName               中文名称
2820  HK_StockArchives     ChiName               中文名称


与实体 '全称变更' 相关的table & column：
        table_name   column_name column_description
79     LC_Business  ChangeReason             简称变更原因
460   LC_ShareStru  ChangeReason             简称变更原因
60   LC_NameChange    ChangeDate             全称更改日期


与实体 'A股简称' 相关的table & column：
          table_name column_name column_description
29  LC_StockArchives  AStockCode             A股证券代码
28  LC_StockArchives  AShareAbbr             A股证券简称
30  LC_StockArchives  BShareAbbr             B股证券简称


与实体 '法人' 相关的table & column：
                   table_name column_name column_description
21           LC_StockArchives   LegalRepr               法人代表
398           LC_Mshareholder   LegalRepr               法人代表
3190  MF_InvestAdvisorOutline   LegalRepr               法人代表
